In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow_datasets as tfds

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [3]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

In [4]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [6]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /Users/wangj3475/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [7]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=0.1.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    urls=['http://ai.stanford.edu/~amaas/data/sentiment/'],
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word V

In [10]:
train_size = info.splits["train"].num_examples

25000

In [11]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ") # line break
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ") # remove all non-letter
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch # all have the same length

In [12]:
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [36]:
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))
vocabulary.most_common()[:3]

[(b'<pad>', 644607), (b'the', 183411), (b'a', 115692)]

In [37]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [38]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [54]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: id=928805, shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

In [56]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

In [60]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  185     2 10865 ...     0     0     0]
 [  718  2409  5629 ...     0     0     0]
 [    6    98     9 ...     0     0     0]
 ...
 [  135     7    73 ...     0     0     0]
 [   22    12     7 ...     0     0     0]
 [10716   438   945 ...     0     0     0]], shape=(32, 66), dtype=int64)
tf.Tensor([1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0], shape=(32,), dtype=int64)


In [57]:
train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [59]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 109s 139ms/step - loss: 0.6444 - accuracy: 0.5846
Epoch 2/5
782/782 [==============================] - 86s 110ms/step - loss: 0.4028 - accuracy: 0.8160
Epoch 3/5
782/782 [==============================] - 117s 150ms/step - loss: 0.2802 - accuracy: 0.8868
Epoch 4/5
782/782 [==============================] - 136s 174ms/step - loss: 0.1906 - accuracy: 0.9292
Epoch 5/5
782/782 [==============================] - 103s 131ms/step - loss: 0.1322 - accuracy: 0.9544


In [62]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None], mask_zero=True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 116s 148ms/step - loss: 0.5368 - accuracy: 0.7241
Epoch 2/5
782/782 [==============================] - 106s 136ms/step - loss: 0.3482 - accuracy: 0.8548
Epoch 3/5
782/782 [==============================] - 156s 199ms/step - loss: 0.1999 - accuracy: 0.9273
Epoch 4/5
782/782 [==============================] - 116s 148ms/step - loss: 0.1359 - accuracy: 0.9520
Epoch 5/5
782/782 [==============================] - 107s 137ms/step - loss: 0.1003 - accuracy: 0.9636


In [ ]:
import tensorflow_hub as hub